<a href="https://colab.research.google.com/github/javadseraj/MLwithTensorflow/blob/master/speech_recognation_farsi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Memory Information

In [ ]:
import psutil
def get_size(bytes, suffix="B"):
    factor = 1024
    for unit in ["", "K", "M", "G", "T", "P"]:
        if bytes < factor:
            return f"{bytes:.2f}{unit}{suffix}"
        bytes /= factor
print("="*40, "Memory Information", "="*40)
svmem = psutil.virtual_memory()
print(f"Total: {get_size(svmem.total)}") ; print(f"Available: {get_size(svmem.available)}")
print(f"Used: {get_size(svmem.used)}") ; print(f"Percentage: {svmem.percent}%")

======================================== Memory Information ========================================
Total: 25.51GB
Available: 24.59GB
Used: 590.48MB
Percentage: 3.6%


# GPU Information

In [ ]:
! nvidia-smi

Mon Aug 24 09:05:43 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import pandas as pd 
import numpy as np 
%pdb
%matplotlib inline
import os
os.chdir('..')
#adress  = '/content/drive/My Drive/deep-data/cv-corpus-5.1-2020-06-22/fa/'
adress="cv-corpus-5.1-2020-06-22/fa/"

Automatic pdb calling has been turned ON


In [ ]:
! wget https://voice-prod-bundler-ee1969a6ce8178826482b88e843c335139bd3fb4.s3.amazonaws.com/cv-corpus-5.1-2020-06-22/fa.tar.gz

--2020-08-24 10:50:25--  https://voice-prod-bundler-ee1969a6ce8178826482b88e843c335139bd3fb4.s3.amazonaws.com/cv-corpus-5.1-2020-06-22/fa.tar.gz
Resolving voice-prod-bundler-ee1969a6ce8178826482b88e843c335139bd3fb4.s3.amazonaws.com (voice-prod-bundler-ee1969a6ce8178826482b88e843c335139bd3fb4.s3.amazonaws.com)... 52.218.230.27
Connecting to voice-prod-bundler-ee1969a6ce8178826482b88e843c335139bd3fb4.s3.amazonaws.com (voice-prod-bundler-ee1969a6ce8178826482b88e843c335139bd3fb4.s3.amazonaws.com)|52.218.230.27|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8470694566 (7.9G) [application/octet-stream]
Saving to: ‘fa.tar.gz’

fa.tar.gz           100%[===================>]   7.89G  22.4MB/s    in 6m 4s   

2020-08-24 10:56:30 (22.2 MB/s) - ‘fa.tar.gz’ saved [8470694566/8470694566]



In [ ]:
import tarfile

In [ ]:
with tarfile.open('fa.tar.gz') as tar:
    tar.extractall()

In [ ]:
!rm fa.tar.gz

rm: cannot remove 'fa.tar.gz': No such file or directory


In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
import tensorflow as tf
print(tf.__version__)
tf.test.is_gpu_available()

1.15.2


True

In [ ]:
EAGER=False
RUN_TESTS=False
PRINT_ALPHABET=False
SAMPLING_RATE=16000

def to_path(filename):
    return   filename


In [ ]:
def random_stretch(audio, params):
    """
    Stretches randomly the input audio
    """
    
    rate = random.uniform(params['random_stretch_min'], params['random_stretch_max'])
    
    return librosa.effects.time_stretch(audio, rate)

In [ ]:
import random

def random_shift(audio, params):
    """
    Shifts randomly the input audio to the left or the right
    """
    
    _shift = random.randrange(params['random_shift_min'], params['random_shift_max'])
    
    if _shift < 0:
        pad = (_shift * -1, 0)
    else:
        pad = (0, _shift)
    
    return np.pad(audio, pad, mode='constant')

In [ ]:
import glob
noise_files = glob.glob(adress + 'clips/*.mp3')
noises = {}

def random_noise(audio, params):
    _factor = random.uniform(
        params['random_noise_factor_min'],
        params['random_noise_factor_max']
    )    
    
    if params['random_noise'] > random.uniform(0, 1):
        
        _path = random.choice(noise_files)
        

        
        ###
        if _path in noises:
            #print('if2')
            wave = noises[_path]
        else:
            #print('else1')
            if os.path.isfile(_path + '.wave.hkl'):
                #print('if3')
                wave = hkl.load(_path + '.wave.hkl').astype(np.float32)
                noises[_path] = wave
            else:
                #print('else2')
                wave, _ = librosa.load(_path, sr=SAMPLING_RATE)
                hkl.dump(wave, _path + '.wave.hkl')
                noises[_path] = wave

        noise = random_shift(
            wave,
            {
                'random_shift_min': -16000,
                'random_shift_max': 16000
            }
        )
        
        max_noise = np.max(noise[0:len(audio)])
        max_wave = np.max(audio)
        
        noise = noise * (max_wave / max_noise)
       
        
        #print('shape noise:', noise.shape)
        #print('audio shape' , audio.shape , len(audio))
        #print('trust noise shape ' ,( noise[0:len(audio)]))

        ###javad
        #if len(noise) <= len(audio) :
          #print("noise len is smaller ")
          #noise  = np.pad( noise  , [(0 , np.abs(len(audio) -len( noise)))])
          #print('shape noise:', noise.shape)
          #print('audio shape' , audio.shape , len(audio))
        #elif len(noise) > len(audio):
          #print("audio len is smaller ")
          #audio = np.pad( audio  , [(0 , np.abs(len(audio) - len(noise)))])
          #print('shape noise:', noise.shape)
          #print('audio shape' , audio.shape , len(audio))
        return _factor * (noise[0:len(audio)] )+ (1.0 - _factor) * audio[0:len(noise)]
    else:
        #print('else3')
        return audio


In [ ]:
"""""
####recode random noise function### 
import glob
noise_files = glob.glob(adress + 'clips/*.mp3')
noises = {}

def random_noise(audio, params):
    _factor = random.uniform(
        params['random_noise_factor_min'],
        params['random_noise_factor_max']
    )    
    
    if params['random_noise'] > random.uniform(0, 1):
        print('if1')
        _path = random.choice(noise_files)
        ##javad

        print(_path)
        ###
        if _path in noises:
            print('if2')
            wave = noises[_path]
        else:
            print('else1')
            if os.path.isfile(_path + '.wave.hkl'):
                print('if3')
                wave = hkl.load(_path + '.wave.hkl').astype(np.float32)
                noises[_path] = wave
            else:
                print('else2')
                wave, _ = librosa.load(_path, sr=SAMPLING_RATE)
                hkl.dump(wave, _path + '.wave.hkl')
                noises[_path] = wave

        noise = random_shift(
            wave,
            {
                'random_shift_min': -16000,
                'random_shift_max': 16000
            }
        )
        
        max_noise = np.max(noise[0:len(audio)])
        max_wave = np.max(audio)
        
        noise = noise * (max_wave / max_noise)
       
        
        print('shape noise:', noise.shape)
        print('audio shape' , audio.shape , len(audio))
        print('trust noise shape ' ,( noise[0:len(audio)]))

        ###javad
        if len(noise) <= len(audio) :
          print("noise len is smaller ")
          noise  = np.pad( noise  , [(0 , np.abs(len(audio) -len( noise)))])
          print('shape noise:', noise.shape)
          print('audio shape' , audio.shape , len(audio))
        elif len(noise) > len(audio):
          print("audio len is smaller ")
          audio = np.pad( audio  , [(0 , np.abs(len(audio) - len(noise)))])
          print('shape noise:', noise.shape)
          print('audio shape' , audio.shape , len(audio))
        return _factor * (noise[0:len(audio)] )+ (1.0 - _factor) * audio
    else:
        print('else3')
        return audio

SyntaxError: ignored

In [ ]:
!pip install hickle

In [ ]:
import hickle as hkl
import librosa
import os.path

def load_wave(example, absolute=False):
    row, params = example
    
    _path =adress+'clips/'+ row.filename if absolute else adress+'clips/'+to_path(row.filename)
   
    
    #print(_path)
    
    
    if os.path.isfile(_path + '.wave.hkl'):
        #print("if111")
        wave = hkl.load(_path + '.wave.hkl').astype(np.float32)
    else:
        #print("else111")
        wave, _ = librosa.load(_path, sr=SAMPLING_RATE)
        hkl.dump(wave,_path + '.wave.hkl')
    if len(wave) <= params['max_wave_length']:
        if params['augment'] :
            wave = random_noise(
                random_stretch(
                    random_shift(
                        wave,
                        params
                    ),
                    params
                ),
                params
            )     
   
  
#
#    else:
#     wave = None
   
    return wave, row

In [ ]:
train_eval_data = df = pd.read_csv('cv-corpus-5.1-2020-06-22/fa/validated.tsv',sep='\t')
df.head()

,client_id,path,sentence,up_votes,down_votes,age,gender,accent,locale,segment
0,016099f3eca09d2769a4bcbef9d90cb9742b9b2bf2b6ad...,common_voice_fa_18325365.mp3,از مهمونداری کنار بکشم,2,1,NaN,NaN,NaN,fa,NaN
1,08c6e3ae4acdced8e01b6cdd184ab85238211761114f12...,common_voice_fa_18960256.mp3,خب ، تو چیكار می كنی؟,2,0,NaN,NaN,NaN,fa,NaN
2,0b8b07282e9b4f9b6495362396d6fd92e35443f3567b4d...,common_voice_fa_21928528.mp3,مسقط پایتخت عمان در عربی به معنای محل سقوط است,2,0,NaN,NaN,NaN,fa,NaN
3,0d02373503174ec83834e30c332811c63bd929c74fbc65...,common_voice_fa_19446941.mp3,آه، نه اصلاُ!,2,0,NaN,NaN,NaN,fa,NaN
4,134a81fa8be98f84ff96f39f37bce55c9d91d96d66dd8b...,common_voice_fa_18557643.mp3,دو استایل متفاوت دارین,2,1,NaN,NaN,NaN,fa,NaN


In [ ]:
#train_eval_data= train_eval_data.rename(columns ={"path": "filename"})
#train_eval_data



In [ ]:
if not os.path.isfile(adress+'train.tsv'):
    eval_data = train_eval_data.sample(n=int(len(train_eval_data) * 0.1 ))
    train_data = train_eval_data[~train_eval_data.isin(eval_data)]
    train_data = train_data.rename(columns ={"path": "filename"})
    train_data = train_data[train_data.filename.notnull()]
    
    train_data.to_csv('train.csv')
    eval_data.to_csv('eval.csv')
else:
  
    train_data = pd.read_csv(adress+'train.tsv'  ,sep ='\t')
    train_data = train_data.rename(columns ={"path": "filename"})
    #eval_data = pd.read_csv(adress+'validated.tsv' , sep ='\t' )
    eval_data = pd.read_csv(adress+'dev.tsv' , sep ='\t' )
    ###add new eval_dataset

In [ ]:
""""
import glob
noise_files = glob.glob(adress + 'clips/*.mp3')
noises = {}

def random_noise(audio, params):
    _factor = random.uniform(
        params['random_noise_factor_min'],
        params['random_noise_factor_max']
    )    
    
    if params['random_noise'] > random.uniform(0, 1):
        print('if1')
        _path = random.choice(noise_files)
        ##javad

        print(_path)
        ###
        if _path in noises:
            print('if2')
            wave = noises[_path]
        else:
            print('else1')
            if os.path.isfile(_path + '.wave.hkl'):
                print('if3')
                wave = hkl.load(_path + '.wave.hkl').astype(np.float32)
                noises[_path] = wave
            else:
                print('else2')
                wave, _ = librosa.load(_path, sr=SAMPLING_RATE)
                hkl.dump(wave, _path + '.wave.hkl')
                noises[_path] = wave

        noise = random_shift(
            wave,
            {
                'random_shift_min': -16000,
                'random_shift_max': 16000
            }
        )
        
        max_noise = np.max(noise[0:len(audio)])
        max_wave = np.max(audio)
        
        noise = noise * (max_wave / max_noise)
        print('shape noise:', noise.shape)
        print('audio shape' , audio.shape , len(audio))
        print('trust noise shape ' ,( noise[0:len(audio)]).shape)
        return _factor * (noise[0:len(audio)] )+ (1.0 - _factor) * audio
    else:
        print('else3')
        return audio

SyntaxError: ignored

In [ ]:
test_data = pd.read_csv(adress+'test.tsv' , sep='\t')

In [ ]:
test_data

,client_id,path,sentence,up_votes,down_votes,age,gender,accent,locale,segment
0,016099f3eca09d2769a4bcbef9d90cb9742b9b2bf2b6ad...,common_voice_fa_18325365.mp3,از مهمونداری کنار بکشم,2,1,NaN,NaN,NaN,fa,NaN
1,08c6e3ae4acdced8e01b6cdd184ab85238211761114f12...,common_voice_fa_18960256.mp3,خب ، تو چیكار می كنی؟,2,0,NaN,NaN,NaN,fa,NaN
2,0b8b07282e9b4f9b6495362396d6fd92e35443f3567b4d...,common_voice_fa_21928528.mp3,مسقط پایتخت عمان در عربی به معنای محل سقوط است,2,0,NaN,NaN,NaN,fa,NaN
3,0d02373503174ec83834e30c332811c63bd929c74fbc65...,common_voice_fa_19446941.mp3,آه، نه اصلاُ!,2,0,NaN,NaN,NaN,fa,NaN
4,134a81fa8be98f84ff96f39f37bce55c9d91d96d66dd8b...,common_voice_fa_18557643.mp3,دو استایل متفاوت دارین,2,1,NaN,NaN,NaN,fa,NaN
...,...,...,...,...,...,...,...,...,...,...
3984,ba6d96cebada9b0847fd1c8a8f67986c03213e9bfef2b6...,common_voice_fa_20623560.mp3,مراقبت کن!,2,0,twenties,female,NaN,fa,NaN
3985,ba6d96cebada9b0847fd1c8a8f67986c03213e9bfef2b6...,common_voice_fa_20623558.mp3,اتاق دو نفره می خواهم.,2,0,twenties,female,NaN,fa,NaN
3986,ba6d96cebada9b0847fd1c8a8f67986c03213e9bfef2b6...,common_voice_fa_20623563.mp3,آیا در هواپیما خانم میهماندار عبری زبان وجود د...,2,0,twenties,female,NaN,fa,NaN
3987,baf063813c603626ae231b57c2abe336217195aa1074e2...,common_voice_fa_18889624.mp3,! ولی رآس هنوز نمیدونه,2,0,NaN,NaN,NaN,fa,NaN


In [ ]:
#train_data['filename'] = train_data['filename'].apply(lambda f: adress_shallow+'/{}'.format(f))

In [ ]:
#adress_shallow = "cv-corpus-5.1-2020-06-22"
adress_shallow = ""
#train_data['filename'] = train_data['filename'].apply(lambda f: adress_shallow+'/{}'.format(f))
train_data['filename'] = train_data['filename'].apply(lambda f: adress_shallow+'/{}'.format(f))
eval_data = eval_data.rename(columns ={"path": "filename"})
test_data = pd.read_csv(adress+'test.tsv' , sep ='\t' )
test_data = test_data.rename(columns ={"path": "filename"})

In [ ]:
eval_data['filename'] = eval_data['filename'].apply(lambda f: adress_shallow+'/{}'.format(f))
test_data['filename'] = test_data['filename'].apply(lambda f: adress_shallow+'/{}'.format(f))

In [ ]:
eval_data

,client_id,filename,sentence,up_votes,down_votes,age,gender,accent,locale,segment
0,ba6f4b705211854faa0ef1044ba9731fe70d4727d27993...,/common_voice_fa_18562452.mp3,خب اون اول چاقو کشید,2,0,NaN,NaN,NaN,fa,NaN
1,ba6f4b705211854faa0ef1044ba9731fe70d4727d27993...,/common_voice_fa_18562451.mp3,میتونم این کار رو بکنم,2,0,NaN,NaN,NaN,fa,NaN
2,ba6f4b705211854faa0ef1044ba9731fe70d4727d27993...,/common_voice_fa_18562454.mp3,روزتون به زیبایی یکی از بهترین سکانس های تاریخ...,2,0,NaN,NaN,NaN,fa,NaN
3,ba9239f9951988085e439f4bce5db6ef272d29a6b355b1...,/common_voice_fa_21577521.mp3,تقریباً هر منطقهای از جهان نوشیدنی تخمیر شده خ...,2,0,NaN,NaN,NaN,fa,NaN
4,ba9239f9951988085e439f4bce5db6ef272d29a6b355b1...,/common_voice_fa_21577523.mp3,در یک ساعت آینده اتفاقی قابل توجه رخ خواهد داد,2,1,NaN,NaN,NaN,fa,NaN
...,...,...,...,...,...,...,...,...,...,...
3984,9d20ea84de42b88dac315b2c62e2951a6f14c027fdf751...,/common_voice_fa_19868205.mp3,در ذهن تو چیست؟,2,0,thirties,male,NaN,fa,NaN
3985,9d20ea84de42b88dac315b2c62e2951a6f14c027fdf751...,/common_voice_fa_19868208.mp3,ما حراج کرده ایم.,2,1,thirties,male,NaN,fa,NaN
3986,9d20ea84de42b88dac315b2c62e2951a6f14c027fdf751...,/common_voice_fa_19868206.mp3,آيا تخفیف دارد؟,2,0,thirties,male,NaN,fa,NaN
3987,a054e15630d582bc4d8c9c28e1d45bc9c627845c2edd82...,/common_voice_fa_21358183.mp3,به این زودی نگو دیره به این زودی نگو بدرود,2,0,NaN,NaN,NaN,fa,NaN


In [ ]:

"""
if not os.path.isfile('full_train.csv'):
    train_data = train_data[['filename', 'text']].append(libri_frame).append(vox_frame)
    train_data.to_csv('full_train.csv')
else:
    train_data = pd.read_csv('full_train.csv') 

"""

"\nif not os.path.isfile('full_train.csv'):\n    train_data = train_data[['filename', 'text']].append(libri_frame).append(vox_frame)\n    train_data.to_csv('full_train.csv')\nelse:\n    train_data = pd.read_csv('full_train.csv') \n\n"

In [ ]:
def print_alphabet():
    uniques = set()
    
    data_frames = [
        train_data,
        test_data
    ]
    
    for ix ,  data_frame in enumerate(data_frames):
        print("Gathering unique characters for set no. " +str(ix + 1) + " (of " + str(len(data_frames)) + " all)") 

        data_frame.apply(lambda row: [ c for c in row.text ], axis=1).apply(uniques.update)
        
    print(sorted(uniques))
    print(''.join(sorted(uniques)))

if PRINT_ALPHABET:
    print_alphabet()

In [ ]:
#rename column sentence to text
train_data =train_data.rename(columns ={"sentence": "text"})
test_data = test_data.rename(columns ={"sentence": "text"})
eval_data = eval_data.rename(columns ={"sentence": "text"})


In [ ]:
 Alpha_bet = print_alphabet()

Gathering unique characters for set no. 1 (of 2 all)
Gathering unique characters for set no. 2 (of 2 all)
[' ', '!', '"', '#', '&', '(', ')', ',', '-', '.', ':', ';', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'M', 'Q', 'S', 'T', 'U', 'Z', '_', 'e', 'm', 'n', 'o', 'u', 'y', '«', '»', '،', '؛', '؟', 'ء', 'آ', 'أ', 'ؤ', 'ئ', 'ا', 'ب', 'ت', 'ث', 'ج', 'ح', 'خ', 'د', 'ذ', 'ر', 'ز', 'س', 'ش', 'ص', 'ض', 'ط', 'ظ', 'ع', 'غ', 'ـ', 'ف', 'ق', 'ك', 'ل', 'م', 'ن', 'ه', 'و', 'ى', 'ي', 'ً', 'ٌ', 'َ', 'ُ', 'ِ', 'ّ', 'ٔ', '٬', 'پ', 'چ', 'ژ', 'ک', 'گ', 'ی', '–', '…', 'ﭘ', 'ﮐ', 'ﮔ', 'ﯽ', 'ﯾ', 'ﯿ', 'ﺍ', 'ﺎ', 'ﺑ', 'ﺒ', 'ﺖ', 'ﺘ', 'ﺧ', 'ﺩ', 'ﺪ', 'ﺭ', 'ﺮ', 'ﺱ', 'ﺴ', 'ﺷ', 'ﺸ', 'ﻀ', 'ﻋ', 'ﻌ', 'ﻟ', 'ﻡ', 'ﻢ', 'ﻤ', 'ﻥ', 'ﻧ', 'ﻪ', 'ﻭ', 'ﻮ']
 !"#&(),-.:;ABCDEFGHIKMQSTUZ_emnouy«»،؛؟ءآأؤئابتثجحخدذرزسشصضطظعغـفقكلمنهوىئًٌَُِّ٬پچژکگی–…ﭘﮐﮔﯽﯾﯿﺍﺎﺑﺒﺖﺘﺧﺩﺪﺭﺮﺱﺴﺷﺸﻀﻋﻌﻟﻡﻢﻤﻥﻧﻪﻭﻮ


In [ ]:
def compute_lengths(original_lengths, params):
    """
    Computes the length of data for CTC
    """
    
    return tf.cast(
        tf.floor(
            (tf.cast(original_lengths, dtype=tf.float32) - params['n_fft']) /
                params['frame_step']
        ) + 1,
        tf.int32
    )

In [ ]:
def encode_labels(labels, params):
    characters = list(params['alphabet'])
    
    table = tf.contrib.lookup.HashTable(
        tf.contrib.lookup.KeyValueTensorInitializer(
            characters,
            list(range(len(characters)))
        ),

        0,#working....
        #-1 ,
        name='char2id'
    )
    
    return table.lookup(
        ### add $ to delimiter
        tf.string_split(labels, delimiter='$')
    )

def decode_codes(codes, params):
    characters = list(params['alphabet'])
    
    table = tf.contrib.lookup.HashTable(
        tf.contrib.lookup.KeyValueTensorInitializer(
            list(range(len(characters))),
            characters
        ),
        '',
        name='id2char'
    )
    
    return table.lookup(codes)

In [ ]:
def decode_logits(logits, lengths, params):
    if len(tf.shape(lengths).shape) == 1:
        lengths = tf.reshape(lengths, [1])
    else:
        lengths = tf.squeeze(lengths)
        
    predicted_codes, _ = tf.nn.ctc_beam_search_decoder(
        tf.transpose(logits, (1, 0, 2)),
        lengths,
        merge_repeated=True
    )
    
    codes = tf.cast(predicted_codes[0], tf.int32)
    
    text = decode_codes(codes, params)
    
    return text, codes


In [ ]:
class LogMelSpectrogram(tf.layers.Layer):
    def __init__(self,
                 sampling_rate,
                 n_fft,
                 frame_step,
                 lower_edge_hertz,
                 upper_edge_hertz,
                 num_mel_bins,
                 **kwargs):
        super(LogMelSpectrogram, self).__init__(**kwargs)
        
        self.sampling_rate = sampling_rate
        self.n_fft = n_fft
        self.frame_step = frame_step
        self.lower_edge_hertz = lower_edge_hertz
        self.upper_edge_hertz = upper_edge_hertz
        self.num_mel_bins = num_mel_bins
        
    def call(self, inputs, training=True):
        stfts = tf.contrib.signal.stft(
            inputs,
            frame_length=self.n_fft,
            frame_step=self.frame_step,
            fft_length=self.n_fft,
            pad_end=False
        )
        
        power_spectrograms = tf.real(stfts * tf.conj(stfts))
        
        num_spectrogram_bins = power_spectrograms.shape[-1].value
    
        linear_to_mel_weight_matrix = tf.constant(
            np.transpose(
                librosa.filters.mel(
                    sr=self.sampling_rate,
                    n_fft=self.n_fft + 1,
                    n_mels=self.num_mel_bins,
                    fmin=self.lower_edge_hertz,
                    fmax=self.upper_edge_hertz
                )
            ),
            dtype=tf.float32
        )
        
        mel_spectrograms = tf.tensordot(
            power_spectrograms,
            linear_to_mel_weight_matrix,
            1
        )
        
        mel_spectrograms.set_shape(
            power_spectrograms.shape[:-1].concatenate(
                linear_to_mel_weight_matrix.shape[-1:]
            )
        )
   
        return tf.log(mel_spectrograms + 1e-6)

In [ ]:
''''stack_kernel_size=7,
stack_filters=3*128,
n_fft=160*8,
frame_step=160*4,
num_mel_bins=160,
optimizer='Momentum',
lr=0.00001,
clip_gradients=20.0
Logmell = LogMelSpectrogram(
            sampling_rate=16000,
            n_fft=n_fft,
            frame_step=frame_step,
            lower_edge_hertz=1000,
            upper_edge_hertz=3000,
            num_mel_bins=160)
         
        """"


SyntaxError: ignored

In [ ]:
class AtrousConv1D(tf.layers.Layer):
    def __init__(self,
                 filters,
                 kernel_size,
                 dilation_rate,
                 use_bias=True,
                 kernel_initializer=tf.glorot_normal_initializer(),
                 causal=True
                ):
        super(AtrousConv1D, self).__init__()
        
        self.filters = filters
        self.kernel_size = kernel_size
        self.dilation_rate = dilation_rate
        self.causal = causal
        
        self.conv1d = tf.layers.Conv1D(
            filters=filters,
            kernel_size=kernel_size,
            dilation_rate=dilation_rate,
            padding='valid' if causal else 'same',
            use_bias=use_bias,
            kernel_initializer=kernel_initializer
        )
        
    def call(self, inputs):
        if self.causal:
            padding = (self.kernel_size - 1) * self.dilation_rate
            inputs = tf.pad(inputs, tf.constant([(0, 0,), (1, 0), (0, 0)]) * padding)
        
        return self.conv1d(inputs)

In [ ]:
class ResidualBlock(tf.layers.Layer):
    def __init__(self, filters, kernel_size, dilation_rate, causal, **kwargs):
        super(ResidualBlock, self).__init__(**kwargs)
        
        self.dilated_conv1 = AtrousConv1D(
            filters=filters,
            kernel_size=kernel_size,
            dilation_rate=dilation_rate,
            causal=causal
        )
        
        self.dilated_conv2 = AtrousConv1D(
            filters=filters,
            kernel_size=kernel_size,
            dilation_rate=dilation_rate,
            causal=causal
        )
        
        self.out = tf.layers.Conv1D(
            filters=filters,
            kernel_size=1
        )
        
    def call(self, inputs, training=True):
        data = tf.layers.batch_normalization(
            inputs,
            training=training
        )
        
        filters = self.dilated_conv1(data)
        gates = self.dilated_conv2(data)
        
        filters = tf.nn.tanh(filters)
        gates = tf.nn.sigmoid(gates)
        
        out = tf.nn.tanh(
            self.out(
                filters * gates
            )
        )
        
        return out + inputs, out

In [ ]:
class ResidualStack(tf.layers.Layer):
    def __init__(self, filters, kernel_size, dilation_rates, causal, **kwargs):
        super(ResidualStack, self).__init__(**kwargs)
      
        self.blocks = [
            ResidualBlock(
                filters=filters,
                kernel_size=kernel_size,
                dilation_rate=dilation_rate,
                causal=causal
            )
            for dilation_rate in dilation_rates
        ]
        
    def call(self, inputs, training=True):
        data = inputs
        skip = 0
        
        for block in self.blocks:
            data, current_skip = block(data, training=training)
            skip += current_skip

        return skip

In [ ]:
class SpeechNet(tf.layers.Layer):
    def __init__(self, params, **kwargs):
        super(SpeechNet, self).__init__(**kwargs)
        
        self.to_log_mel = LogMelSpectrogram(
            sampling_rate=params['sampling_rate'],
            n_fft=params['n_fft'],
            frame_step=params['frame_step'],
            lower_edge_hertz=params['lower_edge_hertz'],
            upper_edge_hertz=params['upper_edge_hertz'],
            num_mel_bins=params['num_mel_bins']
        )
        
        self.expand = tf.layers.Conv1D(
            filters=params['stack_filters'],
            kernel_size=1,
            padding='same'
        )
        
        self.stacks = [
            ResidualStack(
                filters=params['stack_filters'],
                kernel_size=params['stack_kernel_size'],
                dilation_rates=params['stack_dilation_rates'],
                causal=params['causal_convolutions']
            )
            for _ in range(params['stacks'])
        ]
        
        self.out = tf.layers.Conv1D(
            filters=len(params['alphabet']) + 1,
            kernel_size=1,
            padding='same'
        )
        
    def call(self, inputs, training=True):
        data = self.to_log_mel(inputs)
        
        data = tf.layers.batch_normalization(
            data,
            training=training
        )
        
        if len(data.shape) == 2:
            data = tf.expand_dims(data, 0)
        
        data = self.expand(data)
        
        for stack in self.stacks:
            data = stack(data, training=training)
        
        data = tf.layers.batch_normalization(
            data,
            training=training
        )
        
        return self.out(data) + 1e-8

In [ ]:
from multiprocessing import Pool

def input_fn(input_dataset, params, load_wave_fn=load_wave):

    ###global var
    global dataset
    def _input_fn():
        """
        Returns raw audio wave along with the label
        """
         ###global var
        global dataset
        dataset = input_dataset
        
        print(params)
        
        if 'max_text_length' in params and params['max_text_length'] is not None:
            print('Constraining dataset to the max_text_length')
            dataset = input_dataset[input_dataset.text.str.len() < params['max_text_length']]
            
        if 'min_text_length' in params and params['min_text_length'] is not None:
            print('Constraining dataset to the min_text_length')
            dataset = input_dataset[input_dataset.text.str.len() >= params['min_text_length']]
            
        if 'max_wave_length' in params and params['max_wave_length'] is not None:
            print('Constraining dataset to the max_wave_length')
            
        print('Resulting dataset length: {}'.format(len(dataset)))
        
        def generator_fn():
            pool = Pool()
            buffer = []
            ###global var
            global dataset
          
            for epoch in range(params['epochs']):
                
                if params['shuffle']:
                    dataset = dataset.sample(frac=1)
                else:
                    dataset = input_dataset
                    
                for _, row in dataset.iterrows():
                    buffer.append((row, params))

                    if len(buffer) >= params['batch_size']:

                        if params['parallelize']:
                            audios = pool.map(
                                load_wave_fn,
                                buffer
                            )
                        else:
                            audios = map(
                                load_wave_fn,
                                buffer
                            )

                        for audio, row in audios:
                            if audio is not None:
                                if np.isnan(audio).any():
                                    print('SKIPPING! NaN coming from the pipeline!')
                                else:
                                    #print(row.text)
                                    yield (audio, len(audio)), row.text.encode()

                        buffer = []

        return tf.data.Dataset.from_generator(
                generator_fn,
                output_types=((tf.float32, tf.int32), (tf.string)),
                output_shapes=((None,()), (()))
            ) \
            .padded_batch(
                batch_size=params['batch_size'],
                padded_shapes=(
                    (tf.TensorShape([None]), tf.TensorShape(())),
                    tf.TensorShape(())
                )
            )
    
    return _input_fn

In [ ]:
def model_fn(features, labels, mode, params):
    if isinstance(features, dict):
        audio = features['audio']
        original_lengths = features['length']
    else:
        audio, original_lengths = features

    lengths = compute_lengths(original_lengths, params)
    
    if labels is not None:
        codes = encode_labels(labels, params)

    network = SpeechNet(params)

    is_training = mode==tf.estimator.ModeKeys.TRAIN
    
    print('Is training? {}'.format(is_training))

    logits = network(audio, training=is_training)
    text, predicted_codes = decode_logits(logits, lengths, params)

    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions = {
            'logits': logits,
            'text': tf.sparse_tensor_to_dense(
                text,
                ''
            )
        }

        export_outputs = {
            'predictions': tf.estimator.export.PredictOutput(predictions)
        }

        return tf.estimator.EstimatorSpec(
            mode,
            predictions=predictions,
            export_outputs=export_outputs
        )
    else:        
        loss = tf.reduce_mean(
            tf.nn.ctc_loss(
                labels=codes,
                inputs=logits,
                sequence_length=lengths,
                time_major=False,
                ignore_longer_outputs_than_inputs=True
            )
        )

        mean_edit_distance = tf.reduce_mean(
            tf.edit_distance(
                tf.cast(predicted_codes, tf.int32),
                codes
            )
        )

        distance_metric = tf.metrics.mean(mean_edit_distance)

        if mode == tf.estimator.ModeKeys.EVAL:            
            return tf.estimator.EstimatorSpec(
                mode,
                loss=loss,
                eval_metric_ops={ 'edit_distance': distance_metric }
            )

        elif mode == tf.estimator.ModeKeys.TRAIN:
            global_step = tf.train.get_or_create_global_step()

            tf.summary.text(
                'train_predicted_text',
                tf.sparse_tensor_to_dense(text, '')
            )
            tf.summary.scalar('train_edit_distance', mean_edit_distance)

            update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
            with tf.control_dependencies(update_ops):
                train_op = tf.contrib.layers.optimize_loss(
                    loss=loss,
                    global_step=global_step,
                    learning_rate=params['lr'],
                    optimizer=(params['optimizer']),
                    update_ops=update_ops,
                    clip_gradients=params['clip_gradients'],
                    summaries=[
                        "learning_rate",
                        "loss",
                        "global_gradient_norm",
                    ]
                )

            return tf.estimator.EstimatorSpec(
                mode,
                loss=loss,
                train_op=train_op
            )

In [ ]:
def experiment_name(params, excluded_keys=['alphabet', 'data', 'lr', 'clip_gradients']):

    def represent(key, value):
        if key in excluded_keys:
            return None
        else:
            if isinstance(value, list):
                return '{}_{}'.format(key, '_'.join([str(v) for v in value]))
            else:
                return '{}_{}'.format(key, value)

    parts = filter(
        lambda p: p is not None,
        [
            represent(k, params[k])
            for k in sorted(params.keys())
        ]
    )

    return '/'.join(parts)

In [ ]:
def dataset_params(batch_size=32,
                   epochs=50000,
                   parallelize=True,
                   max_text_length=None,
                   min_text_length=None,
                   max_wave_length=80000,
                   shuffle=True,
                   random_shift_min=-4000,
                   random_shift_max= 4000,
                   random_stretch_min=0.7,
                   random_stretch_max= 1.3,
                   random_noise=0.75,
                   random_noise_factor_min=0.2,
                   random_noise_factor_max=0.5,
                   augment=False):
    return {
        'parallelize': parallelize,
        'shuffle': shuffle,
        'max_text_length': max_text_length,
        'min_text_length': min_text_length,
        'max_wave_length': max_wave_length,
        'random_shift_min': random_shift_min,
        'random_shift_max': random_shift_max,
        'random_stretch_min': random_stretch_min,
        'random_stretch_max': random_stretch_max,
        'random_noise': random_noise,
        'random_noise_factor_min': random_noise_factor_min,
        'random_noise_factor_max': random_noise_factor_max,
        'epochs': epochs,
        'batch_size': batch_size,
        'augment': augment
    }

In [ ]:
def experiment_params(data,
                      optimizer='Adam',
                      lr=1e-4,
                      alphabet ='!"#&(),-.:;ABCDEFGHIKMQSTUZ_emnouy«»،؛؟ءآأؤئابتثجحخدذرزسشصضطظعغـفقكلمنهوىئًٌَُِّ٬پچژکگی–…ﭘﮐﮔﯽﯾﯿﺍﺎﺑﺒﺖﺘﺧﺩﺪﺭﺮﺱﺴﺷﺸﻀﻋﻌﻟﻡﻢﻤﻥﻧﻪﻭﻮ',

                      #alphabet=" 'abcdefghijklmnopqrstuvwxyz",
                      causal_convolutions=True,
                      stack_dilation_rates= [1, 3, 9, 27, 81],
                      stacks=2,
                      stack_kernel_size= 3,
                      stack_filters= 32,
                      sampling_rate=16000,
                      n_fft=160*4,
                      frame_step=160,
                      lower_edge_hertz=0,
                      upper_edge_hertz=8000,
                      num_mel_bins=160,
                      clip_gradients=None,
                      codename='regular',
                      **kwargs):
    params = {
        'optimizer': optimizer,
        'lr': lr,
        'data': data,
        'alphabet': alphabet,
        'causal_convolutions': causal_convolutions,
        'stack_dilation_rates': stack_dilation_rates,
        'stacks': stacks,
        'stack_kernel_size': stack_kernel_size,
        'stack_filters': stack_filters,
        'sampling_rate': sampling_rate,
        'n_fft': n_fft,
        'frame_step': frame_step,
        'lower_edge_hertz': lower_edge_hertz,
        'upper_edge_hertz': upper_edge_hertz,
        'num_mel_bins': num_mel_bins,
        'clip_gradients': clip_gradients,
        'codename': codename
    }
    
    #import pdb; pdb.set_trace()
    
    if kwargs is not None and 'data' in kwargs:
        params['data'] = { **params['data'], **kwargs['data'] }
        del kwargs['data']
        
    if kwargs is not None:
        params = { **params, **kwargs }
        
    return params

In [ ]:
import copy

def experiment(data_params=dataset_params(), **kwargs):
    params = experiment_params(
        data_params,
        **kwargs
    )
    
    print(params)

    estimator = tf.estimator.Estimator(
        model_fn=model_fn,
        #model_dir='/content/drive/My Drive/Colab Notebooks' ,
        model_dir='stats/{}'.format(experiment_name(params)),
        params=params
    )
    
    #import pdb; pdb.set_trace()
    
    train_spec = tf.estimator.TrainSpec(
        input_fn=input_fn(
            train_data,
            params['data']
        )
    )
    
    features = {
        "audio": tf.placeholder(dtype=tf.float32, shape=[None]),
        "length": tf.placeholder(dtype=tf.int32, shape=[])
    }
    
    serving_input_receiver_fn = tf.estimator.export.build_raw_serving_input_receiver_fn(
        features
    )
    
    best_exporter = tf.estimator.BestExporter(
      name="best_exporter",
      serving_input_receiver_fn=serving_input_receiver_fn,
      exports_to_keep=5
    )
    
    eval_params = copy.deepcopy(params['data'])
    eval_params['augment'] = False
    
    eval_spec = tf.estimator.EvalSpec(
        input_fn=input_fn(
            eval_data,
            eval_params
        ),
        throttle_secs=60*30,
        exporters=best_exporter
    )
    
    tf.estimator.train_and_evaluate(
        estimator,
        train_spec,
        eval_spec
    )

In [ ]:
def test(data_params=dataset_params(), **kwargs):
    params = experiment_params(
        data_params,
        **kwargs
    )
    
    print(params)

    estimator = tf.estimator.Estimator(
        model_fn=model_fn,
        model_dir='stats/{}'.format(experiment_name(params)),
        params=params
    )
    
    eval_params = copy.deepcopy(params['data'])
    eval_params['augment'] = False
    eval_params['epochs'] = 1
    eval_params['shuffle'] = False

    estimator.evaluate(
        input_fn=input_fn(
            test_data,
            eval_params
        )
    )

In [ ]:
def predict_test(**kwargs):
    params = experiment_params(
        dataset_params(
            augment=False,
            shuffle=False,
            batch_size=1,
            epochs=1,
            parallelize=False
        ),
        **kwargs
    )
    
    print(len(test_data))

    estimator = tf.estimator.Estimator(
        model_fn=model_fn,
        model_dir='stats/{}'.format(experiment_name(params)),
        params=params
    )

    return list(
        estimator.predict(
            input_fn=input_fn(
                test_data,
                params['data']
            )
        )
    )

In [ ]:
def predict(filepath, **kwargs):
    params = experiment_params(
        dataset_params(
            augment=False,
            shuffle=False,
            batch_size=1,
            epochs=1,
            parallelize=False
        ),
        **kwargs
    )
    ###global var
    global dataset
    dataset = pd.DataFrame(columns=['filename', 'text'])
    dataset['filename'] = [filepath]
    dataset['text'] = ['']
    
    print(len(dataset))

    estimator = tf.estimator.Estimator(
        model_fn=model_fn,
        model_dir='stats/{}'.format(experiment_name(params)),
        params=params
    )

    return list(
        estimator.predict(
            input_fn=input_fn(
                dataset,
                params['data']
            )
        )
    )

In [ ]:
def eager_experiment(data_params=dataset_params(), **kwargs):
    params = experiment_params(
        data_params,
        **kwargs
    )
    
    print(params)

    dataset=input_fn(
        train_data,
        params['data']
    )()
    
    features, labels = dataset.make_one_shot_iterator().get_next()
    
    model_fn(features, labels, tf.estimator.ModeKeys.TRAIN, params)

In [ ]:

#dataset=input_fn(
#        train_data,
#        params['data']
#    )()
#    
#features, labels = dataset.make_one_shot_iterator().get_next()

In [ ]:
!pip install hypothesis

In [ ]:
import unittest
from hypothesis import given, settings, note, assume, reproduce_failure
import hypothesis.strategies as st
import hypothesis.extra.numpy as npst

# assuming test path will look like: 1/file.wav
def dummy_load_wave(example):
    row, params = example
    path = row.filename
    
    return np.ones((SAMPLING_RATE)) * float(path.split('/')[0]), row

class TestNotebook(unittest.TestCase):
    
    @given(
        st.sampled_from([22000, 16000, 8000]),
        st.sampled_from([1024, 512]),
        st.sampled_from([1024, 512]),
        npst.arrays(
            np.float32,
            (4, 16000),
            elements=st.floats(-1, 1)
        )
    )
    @settings(max_examples=5)
    def test_log_mel_conversion_works(self, sampling_rate, n_fft, frame_step, audio):
        lower_edge_hertz=0.0
        upper_edge_hertz=sampling_rate / 2.0
        num_mel_bins=64
        
        def librosa_melspectrogram(audio_item):
            spectrogram = np.abs(
                librosa.core.stft(
                    audio_item,
                    n_fft=n_fft,
                    hop_length=frame_step,
                    center=False
                )
            )**2
            
            return np.log(
                librosa.feature.melspectrogram(
                    S=spectrogram,
                    sr=sampling_rate,
                    n_mels=num_mel_bins,
                    fmin=lower_edge_hertz,
                    fmax=upper_edge_hertz,
                ) + 1e-6
            )
            
        audio_ph = tf.placeholder(tf.float32, (4, 16000))
        
        librosa_log_mels = np.transpose(
            np.stack([
                librosa_melspectrogram(audio_item)
                for audio_item in audio
            ]),
            (0, 2, 1)
        )
        
        log_mel_op = tf.check_numerics(
            LogMelSpectrogram(
                sampling_rate=sampling_rate,
                n_fft=n_fft,
                frame_step=frame_step,
                lower_edge_hertz=lower_edge_hertz,
                upper_edge_hertz=upper_edge_hertz,
                num_mel_bins=num_mel_bins
            )(audio_ph),
            message="log mels"
        )
        
        with tf.Session() as session:
            session.run(tf.global_variables_initializer())
            
            log_mels = session.run(
                log_mel_op,
                {
                   audio_ph: audio
                }
            )
            
            np.testing.assert_allclose(
                log_mels,
                librosa_log_mels,
                rtol=1e-1,
                atol=0
            )
    
    @given(
        npst.arrays(
            np.float32,
            (4, 16000),
            elements=st.floats(-1, 1)
        ),
        st.sampled_from([64, 32]),
        st.sampled_from([7, 3]),
        st.sampled_from([1, 4]),
    )
    @settings(max_examples=10)
    def test_residual_block_works(self, audio_waves, filters, size, dilation_rate):
        with tf.Graph().as_default() as g:
            audio_ph = tf.placeholder(tf.float32, (4, None))

            log_mel_op = LogMelSpectrogram(
                sampling_rate=16000,
                n_fft=512,
                frame_step=256,
                lower_edge_hertz=0,
                upper_edge_hertz=8000,
                num_mel_bins=10
            )(audio_ph)

            expanded_op = tf.layers.Dense(filters)(log_mel_op)

            _, block_op = ResidualBlock(
                filters=filters,
                kernel_size=size,
                causal=True,
                dilation_rate=dilation_rate
            )(expanded_op, training=True)

            # really dumb loss function just for the sake
            # of testing:
            loss_op = tf.reduce_sum(block_op)

            variables = tf.trainable_variables()
            self.assertTrue(any(["batch_normalization" in var.name for var in variables]))

            grads_op = tf.gradients(
                loss_op,
                variables
            )
        
            for grad, var in zip(grads_op, variables):
                if grad is None:
                    note(var)

                self.assertTrue(grad is not None)

            with tf.Session(graph=g) as session:
                session.run(tf.global_variables_initializer())

                result, expanded, grads, _ = session.run(
                    [block_op, expanded_op, grads_op, loss_op],
                    {
                        audio_ph: audio_waves
                    }
                )

                self.assertFalse(np.array_equal(result, expanded))
                self.assertEqual(result.shape, expanded.shape)
                self.assertEqual(len(grads), len(variables))
                self.assertFalse(any([np.isnan(grad).any() for grad in grads]))
    
    @given(
        npst.arrays(
            np.float32,
            (4, 16000),
            elements=st.floats(-1, 1)
        ),
        st.sampled_from([64, 32]),
        st.sampled_from([7, 3])
    )
    @settings(max_examples=10)
    def test_residual_stack_works(self, audio_waves, filters, size):
        dilation_rates = [1,2,4]
        
        with tf.Graph().as_default() as g:
            audio_ph = tf.placeholder(tf.float32, (4, None))

            log_mel_op = LogMelSpectrogram(
                sampling_rate=16000,
                n_fft=512,
                frame_step=256,
                lower_edge_hertz=0,
                upper_edge_hertz=8000,
                num_mel_bins=10
            )(audio_ph)

            expanded_op = tf.layers.Dense(filters)(log_mel_op)

            stack_op = ResidualStack(
                filters=filters,
                kernel_size=size,
                causal=True,
                dilation_rates=dilation_rates
            )(expanded_op, training=True)

            # really dumb loss function just for the sake
            # of testing:
            loss_op = tf.reduce_sum(stack_op)

            variables = tf.trainable_variables()
            self.assertTrue(any(["batch_normalization" in var.name for var in variables]))

            grads_op = tf.gradients(
                loss_op,
                variables
            )
        
            for grad, var in zip(grads_op, variables):
                if grad is None:
                    note(var)

                self.assertTrue(grad is not None)

            with tf.Session(graph=g) as session:
                session.run(tf.global_variables_initializer())

                result, expanded, grads, _ = session.run(
                    [stack_op, expanded_op, grads_op, loss_op],
                    {
                        audio_ph: audio_waves
                    }
                )

                self.assertFalse(np.array_equal(result, expanded))
                self.assertEqual(result.shape, expanded.shape)
                self.assertEqual(len(grads), len(variables))
                self.assertFalse(any([np.isnan(grad).any() for grad in grads]))
    
    def test_causal_conv1d_works(self):
        conv_size2_dilation_1 = AtrousConv1D(
            filters=1,
            kernel_size=2,
            dilation_rate=1,
            kernel_initializer=tf.ones_initializer(),
            use_bias=False
        )
        
        conv_size3_dilation_1 = AtrousConv1D(
            filters=1,
            kernel_size=3,
            dilation_rate=1,
            kernel_initializer=tf.ones_initializer(),
            use_bias=False
        )
        
        conv_size2_dilation_2 = AtrousConv1D(
            filters=1,
            kernel_size=2,
            dilation_rate=2,
            kernel_initializer=tf.ones_initializer(),
            use_bias=False
        )
        
        conv_size2_dilation_3 = AtrousConv1D(
            filters=1,
            kernel_size=2,
            dilation_rate=3,
            kernel_initializer=tf.ones_initializer(),
            use_bias=False
        )
        
        data = np.array(list(range(1, 31)))
        data_ph = tf.placeholder(tf.float32, (1, 30, 1))
        
        size2_dilation_1_1 = conv_size2_dilation_1(data_ph)
        size2_dilation_1_2 = conv_size2_dilation_1(size2_dilation_1_1)
        
        size3_dilation_1_1 = conv_size3_dilation_1(data_ph)
        size3_dilation_1_2 = conv_size3_dilation_1(size3_dilation_1_1)
        
        size2_dilation_2_1 = conv_size2_dilation_2(data_ph)
        size2_dilation_2_2 = conv_size2_dilation_2(size2_dilation_2_1)
        
        size2_dilation_3_1 = conv_size2_dilation_3(data_ph)
        size2_dilation_3_2 = conv_size2_dilation_3(size2_dilation_3_1)
        
        with tf.Session() as session:        
            session.run(tf.global_variables_initializer())
            
            outputs = session.run(
                [
                    size2_dilation_1_1,
                    size2_dilation_1_2,
                    size3_dilation_1_1,
                    size3_dilation_1_2,
                    size2_dilation_2_1,
                    size2_dilation_2_2,
                    size2_dilation_3_1,
                    size2_dilation_3_2
                ],
                {
                    data_ph: np.reshape(data, (1, 30, 1))
                }
            )
            
            for ix, out in enumerate(outputs):
                out = np.squeeze(out)
                outputs[ix] = out
                
                self.assertEqual(out.shape[0], len(data))
            
            np.testing.assert_equal(
                outputs[0],
                np.array([1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39, 41, 43, 45, 47, 49, 51, 53, 55, 57, 59], dtype=np.float32)
            )

            np.testing.assert_equal(
                outputs[1],
                np.array([1, 4, 8, 12, 16, 20, 24, 28, 32, 36, 40, 44, 48, 52, 56, 60, 64, 68, 72, 76, 80, 84, 88, 92, 96, 100, 104, 108, 112, 116], dtype=np.float32)
            )

            np.testing.assert_equal(
                outputs[2],
                np.array([1, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60, 63, 66, 69, 72, 75, 78, 81, 84, 87], dtype=np.float32)
            )

            np.testing.assert_equal(
                outputs[3],
                np.array([1, 4, 10, 18, 27, 36, 45, 54, 63, 72, 81, 90, 99, 108, 117, 126, 135, 144, 153, 162, 171, 180, 189, 198, 207, 216, 225, 234, 243, 252], dtype=np.float32)
            )

            np.testing.assert_equal(
                outputs[4],
                np.array([1, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58], dtype=np.float32)
            )

            np.testing.assert_equal(
                outputs[5],
                np.array([1, 2, 5, 8, 12, 16, 20, 24, 28, 32, 36, 40, 44, 48, 52, 56, 60, 64, 68, 72, 76, 80, 84, 88, 92, 96, 100, 104, 108, 112], dtype=np.float32)
            )

            np.testing.assert_equal(
                outputs[6],
                np.array([1, 2, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39, 41, 43, 45, 47, 49, 51, 53, 55, 57], dtype=np.float32)
            )

            np.testing.assert_equal(
                outputs[7],
                np.array([1, 2, 3, 6, 9, 12, 16, 20, 24, 28, 32, 36, 40, 44, 48, 52, 56, 60, 64, 68, 72, 76, 80, 84, 88, 92, 96, 100, 104, 108], dtype=np.float32)
            )
    
    @given(
        npst.arrays(
            np.float32,
            #(4, 30, len('abcdefghijk1234!@#$%^&*')),
            (4, 30, len( '!"#&(),-.:;ABCDEFGHIKMQSTUZ_emnouy«»،؛؟ءآأؤئابتثجحخدذرزسشصضطظعغـفقكلمنهوىئًٌَُِّ٬پچژکگی–…ﭘﮐﮔﯽﯾﯿﺍﺎﺑﺒﺖﺘﺧﺩﺪﺭﺮﺱﺴﺷﺸﻀﻋﻌﻟﻡﻢﻤﻥﻧﻪﻭﻮ')),
            elements=st.floats(0, 1)
        )
    )
    @settings(max_examples=10)
    def test_decode_logits_doesnt_crash(self, logits):
        #params = { 'alphabet': 'abcdefghijk1234!@#$%^&*' }
        params = { 'alphabet' :'!"#&(),-.:;ABCDEFGHIKMQSTUZ_emnouy«»،؛؟ءآأؤئابتثجحخدذرزسشصضطظعغـفقكلمنهوىئًٌَُِّ٬پچژکگی–…ﭘﮐﮔﯽﯾﯿﺍﺎﺑﺒﺖﺘﺧﺩﺪﺭﺮﺱﺴﺷﺸﻀﻋﻌﻟﻡﻢﻤﻥﻧﻪﻭﻮ'}
        lengths = np.array([15, 15, 15, 15], dtype=np.int32)
        
        logits_ph = tf.placeholder(
            tf.float32,
            shape=(4, 30, len(params['alphabet']))
        )
        
        lengths_ph = tf.placeholder(
            tf.int32,
            shape=(4)
        )
        
        decode_op, codes_op = decode_logits(
            logits_ph,
            lengths_ph,
            params
        )
        
        with tf.Session() as session:        
            session.run(tf.global_variables_initializer())
            session.run(tf.tables_initializer(name='init_all_tables'))
            
            codes, decoded = session.run(
                [codes_op, decode_op],
                {
                    logits_ph: logits,
                    lengths_ph: lengths
                }
            )
            
            results = np.ones(codes.dense_shape) * -1
            
            for ix, value in zip(codes.indices, codes.values):
                results[ix[0], ix[1]] = value

            for row in results:
                self.assertLessEqual(len(row[row != -1]), 15)
    ###alphabet
    #@given(st.text(alphabet="!\"&',-.01234:;\ابپتثجچحخدذرزسشصضطظعغفقکگلمنوهی", max_size=10))
    @given(st.text( alphabet ='!"#&(),-.:;ABCDEFGHIKMQSTUZ_emnouy«»،؛؟ءآأؤئابتثجحخدذرزسشصضطظعغـفقكلمنهوىئًٌَُِّ٬پچژکگی–…ﭘﮐﮔﯽﯾﯿﺍﺎﺑﺒﺖﺘﺧﺩﺪﺭﺮﺱﺴﺷﺸﻀﻋﻌﻟﻡﻢﻤﻥﻧﻪﻭﻮ', max_size=10))
    @settings(max_examples=10)
    def test_encode_and_decode_work(self, text):
        assume(text != '')
        ######alphabet
        params = { 'alphabet':'!"#&(),-.:;ABCDEFGHIKMQSTUZ_emnouy«»،؛؟ءآأؤئابتثجحخدذرزسشصضطظعغـفقكلمنهوىئًٌَُِّ٬پچژکگی–…ﭘﮐﮔﯽﯾﯿﺍﺎﺑﺒﺖﺘﺧﺩﺪﺭﺮﺱﺴﺷﺸﻀﻋﻌﻟﻡﻢﻤﻥﻧﻪﻭﻮ' }
        
        label_ph = tf.placeholder(tf.string, shape=(1), name='text')
        codes_op = encode_labels(label_ph, params)
        decode_op = decode_codes(codes_op, params)

        with tf.Session() as session:        
            session.run(tf.global_variables_initializer())
            session.run(tf.tables_initializer(name='init_all_tables'))
            
            codes, decoded = session.run(
                [codes_op, decode_op],
                {
                    label_ph: np.array([text])
                }
            )
            
            note(codes)
            note(decoded)
            
            self.assertEqual(text, ''.join(map(lambda s: s.decode('UTF-8'), decoded.values)))
            self.assertEqual(codes.values.dtype, np.int32)
            self.assertEqual(len(codes.values), len(text))
    
    @given(
        npst.arrays(
            np.float32,
            (st.integers(min_value=16000, max_value=16000*5)),
            elements=st.floats(-1, 1)
        ),
        st.sampled_from([22000, 16000, 8000]),
        st.sampled_from([1024, 512, 640]),
        st.sampled_from([1024, 512, 160]),
    )
    @settings(max_examples=10)
    def test_compute_lengths_works(self,
                                   audio_wave,
                                   sampling_rate,
                                   n_fft,
                                   frame_step
                                  ):
        assume(n_fft >= frame_step)
        
        original_wave_length = audio_wave.shape[0]
        
        audio_waves_ph = tf.placeholder(tf.float32, (None, None), name="audio_waves")
        original_lengths_ph = tf.placeholder(tf.int32, (None), name="original_lengths")
        
        lengths_op = compute_lengths(
            original_lengths_ph,
            {
                'frame_step': frame_step,
                'n_fft': n_fft
            }
        )
        
        self.assertEqual(lengths_op.dtype, tf.int32)
        
        log_mel_op = LogMelSpectrogram(
            sampling_rate=sampling_rate,
            n_fft=n_fft,
            frame_step=frame_step,
            lower_edge_hertz=0.0,
            upper_edge_hertz=8000.0,
            num_mel_bins=13
        )(audio_waves_ph)
        
        with tf.Session() as session:        
            session.run(tf.global_variables_initializer())
            
            lengths, log_mels = session.run(
                [lengths_op, log_mel_op],
                {
                    audio_waves_ph: np.array([audio_wave]),
                    original_lengths_ph: np.array([original_wave_length])
                }
            )
            
            note(original_wave_length)
            note(lengths)
            note(log_mels.shape)
            
            self.assertEqual(lengths[0], log_mels.shape[1])
    
    @given(
        npst.arrays(
            np.float32,
            (4, 16000),
            elements=st.floats(-1, 1)
        )
    )
    @settings(max_examples=10)
    def test_speech_net_works(self, audio_waves):
        with tf.Graph().as_default() as g:
            audio_ph = tf.placeholder(tf.float32, (4, None))

            logits_op = SpeechNet(
                experiment_params(
                    {},
                    stack_dilation_rates= [1, 2, 4],
                    stack_kernel_size= 3,
                    stack_filters= 32,
                    alphabet= 'abcd'
                )
            )(audio_ph)

            # really dumb loss function just for the sake
            # of testing:
            loss_op = tf.reduce_sum(logits_op)

            variables = tf.trainable_variables()
            self.assertTrue(any(["batch_normalization" in var.name for var in variables]))

            grads_op = tf.gradients(
                loss_op,
                variables
            )
        
            for grad, var in zip(grads_op, variables):
                if grad is None:
                    note(var)

                self.assertTrue(grad is not None)

            with tf.Session(graph=g) as session:
                session.run(tf.global_variables_initializer())

                result, grads, _ = session.run(
                    [logits_op, grads_op, loss_op],
                    {
                        audio_ph: audio_waves
                    }
                )

                self.assertEqual(result.shape[2], 5)
                self.assertEqual(len(grads), len(variables))
                self.assertFalse(any([np.isnan(grad).any() for grad in grads]))
                
    @given(
        npst.arrays(
            np.float32,
            (4, 16000),
            elements=st.floats(-1, 1)
        )
    )
    @settings(max_examples=10)
    def test_batch_normalization_when_not_training_doesnt_have_gradients(self, audio_waves):
        with tf.Graph().as_default() as g:
            audio_ph = tf.placeholder(tf.float32, (4, None))

            logits_op = SpeechNet(
                experiment_params(
                    {},
                    stack_dilation_rates= [1, 2, 4],
                    stack_kernel_size= 3,
                    stack_filters= 32,
                    alphabet= 'abcd'
                )
            )(audio_ph, training=False)

            # really dumb loss function just for the sake
            # of testing:
            loss_op = tf.reduce_sum(logits_op)

            variables = tf.trainable_variables()

            grads_op = tf.gradients(
                loss_op,
                variables
            )
        
            for grad, var in zip(grads_op, variables):
                if grad is None:
                    note(var)

                self.assertTrue(grad is not None)

            with tf.Session(graph=g) as session:
                session.run(tf.global_variables_initializer())

                result, grads, _ = session.run(
                    [logits_op, grads_op, loss_op],
                    {
                        audio_ph: audio_waves
                    }
                )
                
                no_batch_norms = list(
                    filter(
                        lambda var: 'batch_normaslization' not in var.name,
                        variables
                    )
                )

                self.assertEqual(len(grads), len(no_batch_norms))
                self.assertFalse(any([np.isnan(grad).any() for grad in grads]))
    
    def test_dataset_returns_data_in_order(self):
        
        params = experiment_params(
            dataset_params(
                batch_size=2,
                epochs=1,
                augment=False
            )
        )
        
        data = pd.DataFrame(
            data={
                'text': [ str(i) for i in range(10)],
                'filename':  [ '{}/wav'.format(i) for i in range(10)]
            }
        )
        
        dataset = input_fn(data, params['data'], dummy_load_wave)()
        iterator = dataset.make_one_shot_iterator()
        next_element = iterator.get_next()
        
        with tf.Session() as session:
            try:
                while True:
                    audio, label = session.run(next_element)
                    audio, length = audio
                    
                    for _audio, _label in zip(list(audio), list(label)):
                        self.assertEqual(_audio[0], float(_label))
                    
                    for _length in length:
                        self.assertEqual(_length, SAMPLING_RATE)
            except tf.errors.OutOfRangeError:
                pass
                

In [ ]:
if __name__ == '__main__' and RUN_TESTS:
    import doctest
    
    doctest.testmod()
    unittest.main(
        argv=['first-arg-is-ignored', 'TestNotebook.test_encode_and_decode_work'],
        failfast=True,
        exit=False
    )

In [ ]:
experiment(
    dataset_params(
        batch_size=18,
        epochs=10,
        # orginall--->epochs=10,
        max_wave_length=320000,
        augment=True,
        random_noise=0.75,
        random_noise_factor_min=0.1,
        random_noise_factor_max=0.15,
        random_stretch_min=0.8,
        random_stretch_max=1.2
    ),
   
     codename='deep_max_20_seconds',
     #alphabet=' !"&\',-.01234:;\\abcdefghijklmnopqrstuvwxyz', # !"&',-.01234:;\abcdefghijklmnopqrstuvwxyz
     alphabet = '!"#&(),-.:;ABCDEFGHIKMQSTUZ_emnouy«»،؛؟ءآأؤئابتثجحخدذرزسشصضطظعغـفقكلمنهوىئًٌَُِّ٬پچژکگی–…ﭘﮐﮔﯽﯾﯿﺍﺎﺑﺒﺖﺘﺧﺩﺪﺭﺮﺱﺴﺷﺸﻀﻋﻌﻟﻡﻢﻤﻥﻧﻪﻭﻮ',
    
     causal_convolutions=False,
     stack_dilation_rates=[1, 3, 9, 27],
     stacks=6,
     stack_kernel_size=7,
     stack_filters=3*128,
     n_fft=160*8,
     frame_step=160*4, 
     num_mel_bins=160,
     optimizer='Momentum',
     lr=0.00001,
     clip_gradients=20.0
     
     )

{'optimizer': 'Momentum', 'lr': 1e-05, 'data': {'parallelize': True, 'shuffle': True, 'max_text_length': None, 'min_text_length': None, 'max_wave_length': 320000, 'random_shift_min': -4000, 'random_shift_max': 4000, 'random_stretch_min': 0.8, 'random_stretch_max': 1.2, 'random_noise': 0.75, 'random_noise_factor_min': 0.1, 'random_noise_factor_max': 0.15, 'epochs': 10, 'batch_size': 18, 'augment': True}, 'alphabet': '!"#&(),-.:;ABCDEFGHIKMQSTUZ_emnouy«»،؛؟ءآأؤئابتثجحخدذرزسشصضطظعغـفقكلمنهوىئًٌَُِّ٬پچژکگی–…ﭘﮐﮔﯽﯾﯿﺍﺎﺑﺒﺖﺘﺧﺩﺪﺭﺮﺱﺴﺷﺸﻀﻋﻌﻟﻡﻢﻤﻥﻧﻪﻭﻮ', 'causal_convolutions': False, 'stack_dilation_rates': [1, 3, 9, 27], 'stacks': 6, 'stack_kernel_size': 7, 'stack_filters': 384, 'sampling_rate': 16000, 'n_fft': 1280, 'frame_step': 640, 'lower_edge_hertz': 0, 'upper_edge_hertz': 8000, 'num_mel_bins': 160, 'clip_gradients': 20.0, 'codename': 'deep_max_20_seconds'}
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmptjxn1plf', '_tf_random_seed': None, '_save_summ

In [ ]:
test(
    dataset_params(
        batch_size=18,
        ##orginal epoch
        #epochs=10,
        epochs = 10 ,
        max_wave_length=320000,
        augment=True,
        random_noise=0.75,
        random_noise_factor_min=0.1,
        random_noise_factor_max=0.15,
        random_stretch_min=0.8,
        random_stretch_max=1.2
    ),
    codename='deep_max_20_seconds',
    alphabet='!"#&(),-.:;ABCDEFGHIKMQSTUZ_emnouy«»،؛؟ءآأؤئابتثجحخدذرزسشصضطظعغـفقكلمنهوىئًٌَُِّ٬پچژکگی–…ﭘﮐﮔﯽﯾﯿﺍﺎﺑﺒﺖﺘﺧﺩﺪﺭﺮﺱﺴﺷﺸﻀﻋﻌﻟﻡﻢﻤﻥﻧﻪﻭﻮ',# !"&',-.01234:;\abcdefghijklmnopqrstuvwxyz
    causal_convolutions=False,
    stack_dilation_rates=[1, 3, 9, 27],
    stacks=6,
    stack_kernel_size=7,
    stack_filters=3*128,
    n_fft=160*8,
    frame_step=160*4,
    num_mel_bins=160,
    optimizer='Momentum',
    lr=0.00001,
    clip_gradients=20.0
)

{'optimizer': 'Momentum', 'lr': 1e-05, 'data': {'parallelize': True, 'shuffle': True, 'max_text_length': None, 'min_text_length': None, 'max_wave_length': 320000, 'random_shift_min': -4000, 'random_shift_max': 4000, 'random_stretch_min': 0.8, 'random_stretch_max': 1.2, 'random_noise': 0.75, 'random_noise_factor_min': 0.1, 'random_noise_factor_max': 0.15, 'epochs': 3, 'batch_size': 18, 'augment': True}, 'alphabet': '!"#&(),-.:;ABCDEFGHIKMQSTUZ_emnouy«»،؛؟ءآأؤئابتثجحخدذرزسشصضطظعغـفقكلمنهوىئًٌَُِّ٬پچژکگی–…ﭘﮐﮔﯽﯾﯿﺍﺎﺑﺒﺖﺘﺧﺩﺪﺭﺮﺱﺴﺷﺸﻀﻋﻌﻟﻡﻢﻤﻥﻧﻪﻭﻮ', 'causal_convolutions': False, 'stack_dilation_rates': [1, 3, 9, 27], 'stacks': 6, 'stack_kernel_size': 7, 'stack_filters': 384, 'sampling_rate': 16000, 'n_fft': 1280, 'frame_step': 640, 'lower_edge_hertz': 0, 'upper_edge_hertz': 8000, 'num_mel_bins': 160, 'clip_gradients': 20.0, 'codename': 'deep_max_20_seconds'}
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'stats/causal_convolutions_False/codename_deep_max_20_sec

In [ ]:
results = predict(
    ##error
    filepath= 'common_voice_fa_18960256.mp3',
    codename='deep_max_20_seconds',
    #alphabet=' !"&\',-.01234:;\\ابپتثجچحخدذرزسشصضطظعغفقکگلمنوهی', # !"&',-.01234:;\abcdefghijklmnopqrstuvwxyz
    alphabet = '!"#&(),-.:;ABCDEFGHIKMQSTUZ_emnouy«»،؛؟ءآأؤئابتثجحخدذرزسشصضطظعغـفقكلمنهوىئًٌَُِّ٬پچژکگی–…ﭘﮐﮔﯽﯾﯿﺍﺎﺑﺒﺖﺘﺧﺩﺪﺭﺮﺱﺴﺷﺸﻀﻋﻌﻟﻡﻢﻤﻥﻧﻪﻭﻮ',
    causal_convolutions=False,
    stack_dilation_rates=[1, 3, 9, 27],
    stacks=6,
    stack_kernel_size=7,
    stack_filters=3*128,
    n_fft=160*8,
    frame_step=160*4,
    num_mel_bins=160,
    optimizer='Momentum',
    lr=0.00001,
    clip_gradients=20.0
)
b''.join(results[0]['text'])


1
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'stats/causal_convolutions_False/codename_deep_max_20_seconds/frame_step_640/lower_edge_hertz_0/n_fft_1280/num_mel_bins_160/optimizer_Momentum/sampling_rate_16000/stack_dilation_rates_1_3_9_27/stack_filters_384/stack_kernel_size_7/stacks_6/upper_edge_hertz_8000', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f

b'y\xd9\x8b\xd8\xb8\xd9\x8by\xe2\x80\xa6\xd8\xa1\xd9\xbe\xd9\x88\xd9\xbe\xe2\x80\xa6\xd8\xb3\xef\xaf\xbf\xd8\xb3\xef\xbb\x9f\xd8\xad\xd8\xb3\xd8\xad\xd8\xb3\xef\xbb\xae\xef\xba\xaaH\xd9\x84\xef\xad\x98\xd8\xb9\xd8\x8c\xd9\x87\xc2\xabI\xd9\x80\xef\xbb\x8b\xd8\xa8\xe2\x80\xa6\xef\xbb\xad\xef\xbb\x9f\xef\xbb\xad\xd8\xb7\xe2\x80\x93Q\xd8\xb5\xda\x98_G\xd8\xb8\xd8\xb7\xd8\xb8'

In [ ]:
test_results = predict_test(
    codename='deep_max_20_seconds',
    #alphabet=' !"&\',-.01234:;\\ابپتثجچحخدذرزسشصضطظعغفقکگلمنوهی', # !"&',-.01234:;\abcdefghijklmnopqrstuvwxyz
    alphabet = '!"#&(),-.:;ABCDEFGHIKMQSTUZ_emnouy«»،؛؟ءآأؤئابتثجحخدذرزسشصضطظعغـفقكلمنهوىئًٌَُِّ٬پچژکگی–…ﭘﮐﮔﯽﯾﯿﺍﺎﺑﺒﺖﺘﺧﺩﺪﺭﺮﺱﺴﺷﺸﻀﻋﻌﻟﻡﻢﻤﻥﻧﻪﻭﻮ',
    causal_convolutions=False,
    stack_dilation_rates=[1, 3, 9, 27],
    stacks=6,
    stack_kernel_size=7,
    stack_filters=3*128,
    n_fft=160*8,
    frame_step=160*4,
    num_mel_bins=160,
    optimizer='Momentum',
    lr=0.00001,
    clip_gradients=20.0
)

In [1]:
#s =b'y\xd9\x8b\xd8\xb8\xd9\x8by\xe2\x80\xa6\xd8\xa1\xd9\xbe\xd9\x88\xd9\xbe\xe2\x80\xa6\xd8\xb3\xef\xaf\xbf\xd8\xb3\xef\xbb\x9f\xd8\xad\xd8\xb3\xd8\xad\xd8\xb3\xef\xbb\xae\xef\xba\xaaH\xd9\x84\xef\xad\x98\xd8\xb9\xd8\x8c\xd9\x87\xc2\xabI\xd9\x80\xef\xbb\x8b\xd8\xa8\xe2\x80\xa6\xef\xbb\xad\xef\xbb\x9f\xef\xbb\xad\xd8\xb7\xe2\x80\x93Q\xd8\xb5\xda\x98_G\xd8\xb8\xd8\xb7\xd8\xb8'
#//print(s.decode('utf-8'))